In [ ]:
def index_to_coordinates(index, num_cols):
    # Convert flattened index to row, column coordinates
    row = index // num_cols
    col = index % num_cols
    return row, col

def coordinates_to_index(row, col, num_cols):
    # Convert row, column coordinates to flattened index
    return row * num_cols + col

def calculate_average(df, new_matrix, index, col_index, num_cols, depth):
    i, j = index_to_coordinates(index, num_cols)

    # Check if max depth is exceeded
    if (
        max(0, i - depth) == 0
        and max(0, j - depth) == 0
        and min(len(df) // num_cols, i + depth + 1) == len(df) // num_cols
        and min(num_cols, j + depth + 1) == num_cols
    ):
        print(f"max depth exceeded for index {index}")
        return

    neighbors_sum = 0
    count = 0

    # Iterate over the neighboring cells
    for x in range(max(0, i - depth), min(len(df) // num_cols, i + depth + 1)):
        for y in range(max(0, j - depth), min(num_cols, j + depth + 1)):
            if x == i and y == j:
                continue
            neighbor_index = coordinates_to_index(x, y, num_cols)

            if pd.notna(df.iloc[neighbor_index][col_index]):
                neighbors_sum += df.iloc[neighbor_index][col_index]
                count += 1

    # Calculate the average
    if count > 0:
        new_matrix.iloc[index, col_index] = neighbors_sum / count
    else:
        calculate_average(df, new_matrix, index, col_index, num_cols, depth + 1)  # If there are no neighbors, keep the current value

    return

def update_matrix(df, num_cols):
    new_matrix = df.copy()

    # Iterate over each cell in the df
    for i in range(len(df)):
        for j in range(len(df.columns)):
            if pd.isna(df.iloc[i, j]):
                # Calculate the average for the current cell
                calculate_average(df, new_matrix, i, j, num_cols, 1)

    return new_matrix


num_cols = 6

group = 1
for df in grouped_dfs:
    updated_matrix = update_matrix(df, num_cols)
    updated_matrix.to_csv(f'data/final-weather/quarterly-interpolated/group_{group}.csv', index=False)
    group += 1